In [8]:
import time
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from mxnet import nd, autograd, gluon
from mxnet.gluon import nn, rnn
import mxnet as mx
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [10]:
df = pd.read_csv(r'../.././Documents/data/TRAINSET_STOCK.csv')
df['vol_change'] = df.groupby(["ts_code"])['vol'].pct_change()
df['vol_change'] = df['vol_change'].fillna(0)
df.head()

,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y,vol_change
0,801010,20140401,农林牧渔,1668.75,1668.54,1689.12,1689.07,22.13,1.33,34914.0,291113.0,41.51,2.77,1,0.000000
1,801010,20140402,农林牧渔,1688.72,1684.53,1693.41,1692.24,3.17,0.19,36300.0,289020.0,41.63,2.79,1,0.039698
2,801010,20140403,农林牧渔,1693.05,1679.85,1697.73,1685.71,-6.53,-0.39,31403.0,259464.0,41.38,2.78,0,-0.134904
3,801010,20140404,农林牧渔,1681.92,1680.34,1698.44,1698.25,12.54,0.74,28648.0,240940.0,41.76,2.80,1,-0.087730
4,801010,20140408,农林牧渔,1693.24,1692.22,1706.84,1706.84,8.59,0.51,35012.0,312423.0,42.00,2.79,1,0.222145


# LSTM

In [13]:
df1 = df[['ts_code','trade_date','name','pct_change','vol_change','y']]
test_df = df.groupby(['trade_date', 'ts_code'])['pct_change']
df2 = df.loc[df1['name']== '农林牧渔']
df2

,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y,vol_change
0,801010,20140401,农林牧渔,1668.75,1668.54,1689.12,1689.07,22.13,1.33,34914.0,291113.0,41.51,2.77,1,0.000000
1,801010,20140402,农林牧渔,1688.72,1684.53,1693.41,1692.24,3.17,0.19,36300.0,289020.0,41.63,2.79,1,0.039698
2,801010,20140403,农林牧渔,1693.05,1679.85,1697.73,1685.71,-6.53,-0.39,31403.0,259464.0,41.38,2.78,0,-0.134904
3,801010,20140404,农林牧渔,1681.92,1680.34,1698.44,1698.25,12.54,0.74,28648.0,240940.0,41.76,2.80,1,-0.087730
4,801010,20140408,农林牧渔,1693.24,1692.22,1706.84,1706.84,8.59,0.51,35012.0,312423.0,42.00,2.79,1,0.222145
5,801010,20140409,农林牧渔,1708.13,1707.49,1726.01,1725.55,18.71,1.10,43114.0,378611.0,41.68,2.81,1,0.231406
6,801010,20140410,农林牧渔,1723.57,1721.18,1730.89,1725.67,0.12,0.01,42089.0,386253.0,41.68,2.81,1,-0.023774
7,801010,20140411,农林牧渔,1721.84,1713.55,1724.08,1719.15,-6.52,-0.38,33973.0,302877.0,41.38,2.80,0,-0.192829
8,801010,20140414,农林牧渔,1718.35,1717.57,1751.81,1749.88,30.73,1.79,45683.0,440762.0,42.11,2.86,1,0.344685
9,801010,20140415,农林牧渔,1747.73,1743.36,1758.92,1748.24,-1.64,-0.09,52524.0,482483.0,41.83,2.89,0,0.149749


In [35]:
import torch.nn as nn



In [ ]:
def creat_dataset(dataset,look_back=10):
    dataX,dataY = [],[]
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i+look_back])
    return np.array(dataX,np.array(dataY))
